In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-csv_2.10:1.3.0 pyspark-shell'

In [2]:
import pyspark
sc = pyspark.SparkContext()

In [3]:
print(sc)

In [4]:
# do something to prove it works
rdd = sc.parallelize(range(1000))
rdd.takeSample(False, 10)

[35, 101, 652, 429, 490, 306, 603, 366, 792, 888]

In [5]:
%pwd

'/home/jovyan/work'

In [6]:
%ls

Dockerfile  Untitled.ipynb


In [7]:
%cd /data

/data


In [8]:
%ls

allen-p     donohoe-t       jones-t          neal-s         skilling-j
arnold-j    dorland-c       kaminski-v       nemec-g        smith-m
arora-h     ermis-f         kean-s           panus-s        south-s
badeer-r    farmer-d        keavey-p         pereira-s      stclair-c
bailey-s    fischer-m       king-j           perlingiere-d  stepenovitch-j
bass-e      fossum-d        kuykendall-t     pimenov-v      storey-g
baughman-d  gay-r           lavorato-j       presto-k       sturm-f
beck-s      germany-c       lay-k            quenet-j       symes-k
benson-r    gilbertsmith-d  lenhart-m        ring-a         taylor-m
brawner-s   giron-d         lewis-a          rodrique-r     tholt-j
buy-r       griffith-j      linder-e         rogers-b       townsend-j
campbell-l  grigsby-m       lokay-m          ruscitti-k     tycholiz-b
carson-m    guzman-m        love-p           sager-e        ward-k
cash-m      haedicke-m      maggi-m          saibi-e        weldon-c
corman-s    hain-m          

In [9]:
import nltk

In [12]:
import json
for person in os.listdir('/data'):
    print(person)


mckay-b
hodge-j
symes-k
whalley-l
king-j
haedicke-m
hendrickson-s
keavey-p
griffith-j
presto-k
maggi-m
dorland-c
ermis-f
donohoe-t
mcconnell-m
corman-s
skilling-j
lay-k
mclaughlin-e
germany-c
cuilla-m
mims-thurston-p
buy-r
storey-g
south-s
schwieger-j
hyvl-d
gilbertsmith-d
sanchez-m
rodrique-r
kean-s
shankman-j
quenet-j
stepenovitch-j
nemec-g
hernandez-j
guzman-m
zipper-a
townsend-j
perlingiere-d
love-p
smith-m
mann-k
fischer-m
bailey-s
dickson-s
merriss-s
dean-c
lenhart-m
fossum-d
davis-d
may-l
martin-t
weldon-c
linder-e
pereira-s
brawner-s
ring-a
horton-s
beck-s
hain-m
panus-s
tholt-j
stclair-c
sanders-r
ward-k
lavorato-j
lewis-a
pimenov-v
heard-m
ruscitti-k
kaminski-v
saibi-e
allen-p
gay-r
hayslett-r
zufferli-j
tycholiz-b
derrick-j
sturm-f
shively-h
neal-s
giron-d
grigsby-m
campbell-l
dasovich-j
lokay-m
williams-j
scott-s
badeer-r
wolfe-j
taylor-m
bass-e
farmer-d
baughman-d
ybarbo-p
shackleton-s
white-s
whalley-g
carson-m
rogers-b
arora-h
arnold-j
cash-m
jones-t
benson-r
kuykendall-